In [ ]:
!gdown 1-79PwVv0lqrl04LQLoBsmqyFou7gu3oO
!gdown 1nn18EtE6h__ijhlNF5ZZ1XMBCl12zJdf

Downloading...
From: https://drive.google.com/uc?id=1-79PwVv0lqrl04LQLoBsmqyFou7gu3oO
To: /content/detectron2-0.6-cp311-cp311-linux_x86_64.whl
100% 6.38M/6.38M [00:00<00:00, 43.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1nn18EtE6h__ijhlNF5ZZ1XMBCl12zJdf
From (redirected): https://drive.google.com/uc?id=1nn18EtE6h__ijhlNF5ZZ1XMBCl12zJdf&confirm=t&uuid=8b95a69f-e239-4668-b867-e830537c233a
To: /content/car_part_model.pth
100% 352M/352M [00:07<00:00, 46.6MB/s]


In [ ]:
!pip install fastapi pyngrok uvicorn python-multipart
!pip install uv
!pip install yacs fvcore iopath==0.1.8 omegaconf hydra-core==1.1 black
!uv pip install detectron2-0.6-cp311-cp311-linux_x86_64.whl --system

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.7 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141214 sha256=e5d08f7041e

Using Python 3.11.11 environment at: /usr
Resolved 39 packages in 246ms
Prepared 1 package in 70ms
Installed 1 package in 7ms
 + detectron2==0.6 (from file:///content/detectron2-0.6-cp311-cp311-linux_x86_64.whl)


In [ ]:
import random
from dataclasses import dataclass
from typing import Any, List, Dict, Optional, Union, Tuple
import cv2
import torch
import requests
import numpy as np
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from transformers import AutoModelForMaskGeneration, AutoProcessor, pipeline
import torch
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import numpy as np
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
import cv2
import numpy as np
from pathlib import Path

@dataclass
class BoundingBox:
    xmin: int
    ymin: int
    xmax: int
    ymax: int

    @property
    def xyxy(self) -> List[float]:
        return [self.xmin, self.ymin, self.xmax, self.ymax]

@dataclass
class DetectionResult:
    score: float
    label: str
    box: BoundingBox
    mask: Optional[np.array] = None

    @classmethod
    def from_dict(cls, detection_dict: Dict) -> 'DetectionResult':
        return cls(score=detection_dict['score'],
                   label=detection_dict['label'],
                   box=BoundingBox(xmin=detection_dict['box']['xmin'],
                                   ymin=detection_dict['box']['ymin'],
                                   xmax=detection_dict['box']['xmax'],
                                   ymax=detection_dict['box']['ymax']))

def annotate(image: Union[Image.Image, np.ndarray], detection_results: List[DetectionResult]) -> np.ndarray:
    # Convert PIL Image to OpenCV format
    image_cv2 = np.array(image) if isinstance(image, Image.Image) else image
    image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_RGB2BGR)

    # Iterate over detections and add bounding boxes and masks
    for detection in detection_results:
        label = detection.label
        score = detection.score
        box = detection.box
        mask = detection.mask

        # Sample a random color for each detection
        color = np.random.randint(0, 256, size=3)

        # Draw bounding box
        cv2.rectangle(image_cv2, (box.xmin, box.ymin), (box.xmax, box.ymax), color.tolist(), 2)
        cv2.putText(image_cv2, f'{label}: {score:.2f}', (box.xmin, box.ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)

        # If mask is available, apply it
        if mask is not None:
            # Convert mask to uint8
            mask_uint8 = (mask * 255).astype(np.uint8)
            contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(image_cv2, contours, -1, color.tolist(), 2)

    return cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)

def plot_detections(
    image: Union[Image.Image, np.ndarray],
    detections: List[DetectionResult],
    save_name: Optional[str] = None
) -> None:
    annotated_image = annotate(image, detections)
    plt.imshow(annotated_image)
    plt.axis('off')
    if save_name:
        plt.savefig(save_name, bbox_inches='tight')
    plt.show()

def random_named_css_colors(num_colors: int) -> List[str]:
    """
    Returns a list of randomly selected named CSS colors.

    Args:
    - num_colors (int): Number of random colors to generate.

    Returns:
    - list: List of randomly selected named CSS colors.
    """
    # List of named CSS colors
    named_css_colors = [
        'aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond',
        'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
        'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey',
        'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
        'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue',
        'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite',
        'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory',
        'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow',
        'lightgray', 'lightgreen', 'lightgrey', 'lightpink', 'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray',
        'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine',
        'mediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise',
        'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 'olive',
        'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 'palevioletred', 'papayawhip',
        'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'rebeccapurple', 'red', 'rosybrown', 'royalblue', 'saddlebrown',
        'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 'slategray', 'slategrey',
        'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white',
        'whitesmoke', 'yellow', 'yellowgreen'
    ]

    # Sample random named CSS colors
    return random.sample(named_css_colors, min(num_colors, len(named_css_colors)))

def plot_detections_plotly(
    image: np.ndarray,
    detections: List[DetectionResult],
    class_colors: Optional[Dict[str, str]] = None
) -> None:
    # If class_colors is not provided, generate random colors for each class
    if class_colors is None:
        num_detections = len(detections)
        colors = random_named_css_colors(num_detections)
        class_colors = {}
        for i in range(num_detections):
            class_colors[i] = colors[i]


    fig = px.imshow(image)

    # Add bounding boxes
    shapes = []
    annotations = []
    for idx, detection in enumerate(detections):
        label = detection.label
        box = detection.box
        score = detection.score
        mask = detection.mask

        polygon = mask_to_polygon(mask)

        fig.add_trace(go.Scatter(
            x=[point[0] for point in polygon] + [polygon[0][0]],
            y=[point[1] for point in polygon] + [polygon[0][1]],
            mode='lines',
            line=dict(color=class_colors[idx], width=2),
            fill='toself',
            name=f"{label}: {score:.2f}"
        ))

        xmin, ymin, xmax, ymax = box.xyxy
        shape = [
            dict(
                type="rect",
                xref="x", yref="y",
                x0=xmin, y0=ymin,
                x1=xmax, y1=ymax,
                line=dict(color=class_colors[idx])
            )
        ]
        annotation = [
            dict(
                x=(xmin+xmax) // 2, y=(ymin+ymax) // 2,
                xref="x", yref="y",
                text=f"{label}: {score:.2f}",
            )
        ]

        shapes.append(shape)
        annotations.append(annotation)

    # Update layout
    button_shapes = [dict(label="None",method="relayout",args=["shapes", []])]
    button_shapes = button_shapes + [
        dict(label=f"Detection {idx+1}",method="relayout",args=["shapes", shape]) for idx, shape in enumerate(shapes)
    ]
    button_shapes = button_shapes + [dict(label="All", method="relayout", args=["shapes", sum(shapes, [])])]

    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=True,
        updatemenus=[
            dict(
                type="buttons",
                direction="up",
                buttons=button_shapes
            )
        ],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Show plot
    fig.show()
def mask_to_polygon(mask: np.ndarray) -> List[List[int]]:
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Extract the vertices of the contour
    polygon = largest_contour.reshape(-1, 2).tolist()

    return polygon

def polygon_to_mask(polygon: List[Tuple[int, int]], image_shape: Tuple[int, int]) -> np.ndarray:
    """
    Convert a polygon to a segmentation mask.

    Args:
    - polygon (list): List of (x, y) coordinates representing the vertices of the polygon.
    - image_shape (tuple): Shape of the image (height, width) for the mask.

    Returns:
    - np.ndarray: Segmentation mask with the polygon filled.
    """
    # Create an empty mask
    mask = np.zeros(image_shape, dtype=np.uint8)

    # Convert polygon to an array of points
    pts = np.array(polygon, dtype=np.int32)

    # Fill the polygon with white color (255)
    cv2.fillPoly(mask, [pts], color=(255,))

    return mask

def load_image(image_str: str) -> Image.Image:
    if image_str.startswith("http"):
        image = Image.open(requests.get(image_str, stream=True).raw).convert("RGB")
    else:
        image = Image.open(image_str).convert("RGB")

    return image

def get_boxes(results: DetectionResult) -> List[List[List[float]]]:
    boxes = []
    for result in results:
        xyxy = result.box.xyxy
        boxes.append(xyxy)

    return [boxes]

def refine_masks(masks: torch.BoolTensor, polygon_refinement: bool = False) -> List[np.ndarray]:
    masks = masks.cpu().float()
    masks = masks.permute(0, 2, 3, 1)
    masks = masks.mean(axis=-1)
    masks = (masks > 0).int()
    masks = masks.numpy().astype(np.uint8)
    masks = list(masks)

    if polygon_refinement:
        for idx, mask in enumerate(masks):
            shape = mask.shape
            polygon = mask_to_polygon(mask)
            mask = polygon_to_mask(polygon, shape)
            masks[idx] = mask

    return masks

def detect(
    image: Image.Image,
    labels: List[str],
    threshold: float = 0.3,
    detector_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Use Grounding DINO to detect a set of labels in an image in a zero-shot fashion.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    detector_id = detector_id if detector_id is not None else "IDEA-Research/grounding-dino-tiny"
    object_detector = pipeline(model=detector_id, task="zero-shot-object-detection", device=device)

    labels = [label if label.endswith(".") else label+"." for label in labels]

    results = object_detector(image,  candidate_labels=labels, threshold=threshold)
    results = [DetectionResult.from_dict(result) for result in results]

    return results

def segment(
    image: Image.Image,
    detection_results: List[Dict[str, Any]],
    polygon_refinement: bool = False,
    segmenter_id: Optional[str] = None
) -> List[DetectionResult]:
    """
    Use Segment Anything (SAM) to generate masks given an image + a set of bounding boxes.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    segmenter_id = segmenter_id if segmenter_id is not None else "facebook/sam-vit-base"

    segmentator = AutoModelForMaskGeneration.from_pretrained(segmenter_id).to(device)
    processor = AutoProcessor.from_pretrained(segmenter_id)

    boxes = get_boxes(detection_results)
    inputs = processor(images=image, input_boxes=boxes, return_tensors="pt").to(device)

    outputs = segmentator(**inputs)
    masks = processor.post_process_masks(
        masks=outputs.pred_masks,
        original_sizes=inputs.original_sizes,
        reshaped_input_sizes=inputs.reshaped_input_sizes
    )[0]

    masks = refine_masks(masks, polygon_refinement)

    for detection_result, mask in zip(detection_results, masks):
        detection_result.mask = mask

    return detection_results

def grounded_segmentation(
    image: Union[Image.Image, str],
    labels: List[str],
    threshold: float = 0.3,
    polygon_refinement: bool = False,
    detector_id: Optional[str] = None,
    segmenter_id: Optional[str] = None
) -> Tuple[np.ndarray, List[DetectionResult]]:
    if isinstance(image, str):
        image = load_image(image)

    detections = detect(image, labels, threshold, detector_id)
    detections = segment(image, detections, polygon_refinement, segmenter_id)

    return np.array(image), detections

# Part category mapping
category_mapping_parts = {
    "Windshield": 0,
    "Back-windshield": 1,
    "Front-window": 2,
    "Back-window": 3,
    "Front-door": 4,
    "Back-door": 5,
    "Front-wheel": 6,
    "Cracked": 7,
    "Front-bumper": 8,
    "Back-bumper": 9,
    "Headlight": 10,
    "Tail-light": 11,
    "Hood": 12,
    "Trunk": 13,
    "License-plate": 14,
    "Mirror": 15,
    "Roof": 16,
    "Grille": 17,
    "Rocker-panel": 18,
    "Quarter-panel": 19,
    "Fender": 20
}

# Reverse mapping for part categories
id_to_part_name = {v: k for k, v in category_mapping_parts.items()}

def load_part_model(model_path, threshold=0.4):
    """
    Load the Detectron2 model for part detection with instance segmentation

    Args:
        model_path (str): Path to the model weights
        threshold (float): Detection confidence threshold

    Returns:
        DefaultPredictor: Loaded Detectron2 model predictor
    """
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 21
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    return DefaultPredictor(cfg)

def detect_car_parts(image_path, model_predictor):
    """
    Detect car parts in an image with instance segmentation masks

    Args:
        image_path (str or numpy.ndarray): Path to the input image or image array
        model_predictor: Loaded Detectron2 predictor

    Returns:
        dict: Dictionary containing detected parts, their masks, and other information
    """
    # Read image
    if isinstance(image_path, str):
        image = cv2.imread(image_path)
    else:
        image = image_path

    if image is None:
        raise ValueError("Could not load image")

    # Run detection
    outputs = model_predictor(image)
    instances = outputs["instances"].to("cpu")

    # Get predictions
    boxes = instances.pred_boxes.tensor.numpy()
    classes = instances.pred_classes.numpy()
    scores = instances.scores.numpy()
    masks = instances.pred_masks.numpy()

    # Prepare results
    results = []
    for box, mask, class_id, score in zip(boxes, masks, classes, scores):
        part_name = id_to_part_name.get(class_id, "Unknown")
        results.append({
            "part": part_name,
            "confidence": float(score),
            "bbox": box.tolist(),  # [x1, y1, x2, y2]
            "mask": mask  # Binary mask for the instance
        })

    return {
        "instances": results,
        "image_size": image.shape[:2]  # (height, width)
    }

def visualize_detections(image_path, results, draw_masks=True):
    """
    Visualize detected car parts on the image with instance segmentation masks

    Args:
        image_path (str or numpy.ndarray): Path to the input image or image array
        results (dict): Detection results from detect_car_parts
        draw_masks (bool): Whether to draw segmentation masks (True) or just boxes (False)

    Returns:
        numpy.ndarray: Image with visualized detections
    """
    if isinstance(image_path, str):
        image = cv2.imread(image_path)
    else:
        image = image_path.copy()

    # Create metadata for visualization
    metadata = MetadataCatalog.get("car_parts_metadata")
    if not metadata:
        metadata = MetadataCatalog.add("car_parts_metadata", {})
        metadata.thing_classes = list(category_mapping_parts.keys())
        metadata.thing_colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                               for _ in range(len(category_mapping_parts))]

    # Convert results back to Detectron2 Instances format for visualization
    instances = results["instances"]

    # Create visualizer
    v = Visualizer(image[:, :, ::-1],
                  metadata=metadata,
                  scale=1.0,
                  instance_mode=ColorMode.IMAGE_BW if draw_masks else ColorMode.SEGMENTATION)

    # Draw predictions
    out = v.draw_instance_predictions(instances)
    return out.get_image()[:, :, ::-1]

def get_mask_overlay(image, mask, color=(0, 255, 0), alpha=0.5):
    """
    Create a transparent overlay of a colored mask on an image

    Args:
        image (numpy.ndarray): Original image
        mask (numpy.ndarray): Binary mask
        color (tuple): RGB color for the mask
        alpha (float): Transparency level (0-1)

    Returns:
        numpy.ndarray: Image with colored mask overlay
    """
    colored_mask = np.zeros_like(image)
    for c in range(3):
        colored_mask[:, :, c] = mask * color[c]

    return cv2.addWeighted(image, 1, colored_mask, alpha, 0)

unpaintable_parts = [
    "Front-wheel",
    "Back-wheel",
    "Windshield",
    "Back-windshield",
    "Front-window",
    "Back-window",
    "Headlight",
    "Tail-light",
    "License-plate",
    "Mirror",
    "Grille"
]

In [ ]:
# image_url = '/content/KARMA INVICTUS.jfif'
# labels = ["car"]
# threshold = 0.8

# detector_id = "IDEA-Research/grounding-dino-tiny"
# segmenter_id = "facebook/sam-vit-base"
# image_array, detections = grounded_segmentation(
#     image=image_url,
#     labels=labels,
#     threshold=threshold,
#     polygon_refinement=True,
#     detector_id=detector_id,
#     segmenter_id=segmenter_id
# )
# mask=detections[0].mask
# model_path = "car_part_model.pth"
# predictor = load_part_model(model_path)

# detections = detect_car_parts(image_url, predictor)
# detections_unpaintable = [detection for detection in detections["instances"] if detection["part"] in unpaintable_parts]
# unpaintable_regions=np.zeros_like(mask)
# for region in detections_unpaintable:
#     region=region['mask'].astype('uint8')
#     unpaintable_regions+=region
# unpaintable_regions = np.where(unpaintable_regions > 1, 1, unpaintable_regions)
# final=unpaintable_regions+mask

In [ ]:
# plt.imshow(final)

In [ ]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, Response
import numpy as np
import cv2
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from PIL import Image
import io
import base64


app = FastAPI(title="Car Mask Generation API")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global variables
MODEL_PATH = "car_part_model.pth"
DETECTOR_ID = "IDEA-Research/grounding-dino-tiny"
SEGMENTER_ID = "facebook/sam-vit-base"
LABELS = ["car"]
THRESHOLD = 0.8

# Load model at startup
predictor = load_part_model(MODEL_PATH)

def process_image(image_bytes):
    """Process the uploaded image and return the mask"""
    try:
        # Convert bytes to numpy array
        nparr = np.frombuffer(image_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Save temporary file for processing
        temp_path = '/tmp/temp_image.jpg'
        cv2.imwrite(temp_path, image)

        # Generate car mask
        image_array, detections = grounded_segmentation(
            image=temp_path,
            labels=LABELS,
            threshold=THRESHOLD,
            polygon_refinement=True,
            detector_id=DETECTOR_ID,
            segmenter_id=SEGMENTER_ID
        )
        mask = detections[0].mask

        # Detect unpaintable parts
        detections = detect_car_parts(temp_path, predictor)
        detections_unpaintable = [
            detection for detection in detections["instances"]
            if detection["part"] in unpaintable_parts
        ]

        # Create unpaintable regions mask
        unpaintable_regions = np.zeros_like(mask)
        for region in detections_unpaintable:
            region = region['mask'].astype('uint8')
            unpaintable_regions += region

        unpaintable_regions = np.where(unpaintable_regions > 1, 1, unpaintable_regions)
        final = unpaintable_regions + mask

        return final

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate_mask")
async def generate_mask(file: UploadFile = File(...)):
    """
    Generate a mask for the uploaded car image
    """
    try:
        # Process image
        print("Content type:", file.content_type)  # Debug print
        print("Filename:", file.filename)  # Debug print

        # Read file content
        contents = await file.read()
        print("File size:", len(contents))
        nparr = np.frombuffer(contents, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        print("Image shape:", image.shape if image is not None else "None")  # Debug print

        if image is None:
            raise HTTPException(
                status_code=400,
                detail="Could not decode image. Please ensure it's a valid image file."
            )

        # Save temporary file for processing
        temp_path = '/tmp/temp_image.jpg'
        cv2.imwrite(temp_path, image)

        # Your existing processing code here
        image_array, detections = grounded_segmentation(
            image=temp_path,
            labels=LABELS,
            threshold=THRESHOLD,
            polygon_refinement=True,
            detector_id=DETECTOR_ID,
            segmenter_id=SEGMENTER_ID
        )

        mask = detections[0].mask

        # Process unpaintable parts
        detections = detect_car_parts(temp_path, predictor)
        detections_unpaintable = [
            detection for detection in detections["instances"]
            if detection["part"] in unpaintable_parts
        ]

        unpaintable_regions = np.zeros_like(mask)
        for region in detections_unpaintable:
            region = region['mask'].astype('uint8')
            unpaintable_regions += region

        unpaintable_regions = np.where(unpaintable_regions > 1, 1, unpaintable_regions)
        final = unpaintable_regions + mask

        # Convert mask to base64
        _, buffer = cv2.imencode('.png', final)
        mask_base64 = base64.b64encode(buffer).decode()

        return {
            "status": "success",
            "mask": mask_base64
        }

    except Exception as e:
        print(e)
        raise HTTPException(
            status_code=500,
            detail=str(e)
        )

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy"}

def start_server():
    """Start the FastAPI server with ngrok tunnel"""
    # Start ngrok tunnel
    ngrok.set_auth_token("2pkpCJEM14i61fW9Up53lbWMOE9_3JvxuVTV5gtiWed6keccm")
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)

    nest_asyncio.apply()
    uvicorn.run(app, port=8000)

if __name__ == "__main__":
    start_server()

/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

INFO:     Started server process [249]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://da6d-34-34-25-54.ngrok-free.app
Content type: image/jpeg
Filename: image.jpg
File size: 41828
Image shape: (450, 800, 3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

...

INFO:     49.43.5.78:0 - "POST /generate_mask HTTP/1.1" 200 OK
Content type: image/jpeg
Filename: image.jpg
File size: 41828
Image shape: (450, 800, 3)


Device set to use cuda


INFO:     49.43.5.78:0 - "POST /generate_mask HTTP/1.1" 200 OK
Content type: image/jpeg
Filename: image.jpg
File size: 189408
Image shape: (580, 1060, 3)


Device set to use cuda


INFO:     49.43.5.78:0 - "POST /generate_mask HTTP/1.1" 200 OK
